In [1]:
from sklearn.datasets import make_friedman1
from sklearn import feature_selection
from sklearn import preprocessing
from sklearn import pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVR

D:\WindowsApps\Anaconda\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\WindowsApps\Anaconda\envs\tensorflow\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)

print(X[:5])
print(y[:4])

[[ 0.5488135   0.71518937  0.60276338  0.54488318  0.4236548   0.64589411
   0.43758721  0.891773    0.96366276  0.38344152]
 [ 0.79172504  0.52889492  0.56804456  0.92559664  0.07103606  0.0871293
   0.0202184   0.83261985  0.77815675  0.87001215]
 [ 0.97861834  0.79915856  0.46147936  0.78052918  0.11827443  0.63992102
   0.14335329  0.94466892  0.52184832  0.41466194]
 [ 0.26455561  0.77423369  0.45615033  0.56843395  0.0187898   0.6176355
   0.61209572  0.616934    0.94374808  0.6818203 ]
 [ 0.3595079   0.43703195  0.6976312   0.06022547  0.66676672  0.67063787
   0.21038256  0.1289263   0.31542835  0.36371077]]
[ 17.2134918   19.37965436  14.75030901  11.81662128]


In [2]:
est = SVR(kernel="linear")

selector = feature_selection.RFE(est)
std_scaler = preprocessing.StandardScaler()
pipe_params = [('std_scaler', std_scaler),('select', selector), ('clf', est)]

class myPipe(pipeline.Pipeline):

    def fit(self, X,y):
        """Calls last elements .coef_ method.
        Based on the sourcecode for decision_function(X).
        Link: https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/pipeline.py
        ----------
        """

        super(myPipe, self).fit(X,y)

        self.coef_=self.steps[-1][-1].coef_
        return

pipe = myPipe(pipe_params)
    
selector = feature_selection.RFE(pipe)
clf = GridSearchCV(selector, param_grid={'estimator__clf__C': [2, 10]})
clf.fit(X, y)

print(clf.best_estimator_)
print(clf.best_params_)

RFE(estimator=myPipe(steps=[('std_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('select', RFE(estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
  n_features_to_select=None, step=1, verbose=0)), ('clf', SVR(C=2, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]),
  n_features_to_select=None, step=1, verbose=0)
{'estimator__clf__C': 2}


In [7]:
?RFECV

In [14]:
>>> from sklearn.datasets import make_friedman1
>>> from sklearn.feature_selection import RFECV
>>> from sklearn.svm import SVR
>>> X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
print(X[:1],y[:1])
>>> estimator = SVR(kernel="linear")
>>> selector = RFECV(estimator, step=1, cv=4)
>>> selector = selector.fit(X, y)
>>> print(selector.support_) # doctest: +NORMALIZE_WHITESPACE
>>> selector.ranking_

[[ 0.5488135   0.71518937  0.60276338  0.54488318  0.4236548   0.64589411
   0.43758721  0.891773    0.96366276  0.38344152]] [ 17.2134918]
[ True  True  True  True  True False False False  True False]


array([1, 1, 1, 1, 1, 5, 3, 2, 1, 4])

In [15]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFECV
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVR
X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
param_grid = [{'C': 0.01}, {'C': 0.1}, {'C': 1.0}, {'C': 10.0}, {'C': 100.0}, {'C': 1000.0}, {'C': 10000.0}]
estimator = SVR(kernel="linear")
selector = RFECV(estimator, step=1, cv=4)
clf = GridSearchCV(selector, {'estimator_params': param_grid}, cv=7)


TypeError: __init__() missing 1 required positional argument: 'param_grid'

In [16]:
clf.fit(X, y)
clf.best_estimator_.estimator_
clf.best_estimator_.grid_scores_
clf.best_estimator_.ranking_

ValueError: Invalid parameter estimator_params for estimator RFECV. Check the list of available parameters with `estimator.get_params().keys()`.

In [19]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFECV
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint as sp_randint

# Build a classification task using 5 informative features
X, y = make_classification(n_samples=1000, n_features=25, n_informative=5, n_redundant=2, n_repeated=0, n_classes=8, n_clusters_per_class=1, random_state=0)

grid = {"estimator__bootstrap": [True, False],
        "estimator__criterion": ["gini", "entropy"]}

estimator = RandomForestClassifier()
selector = RFECV(estimator, step=1, cv=4)
clf = RandomizedSearchCV(selector, param_distributions=grid, cv=7)
clf.fit(X, y)
print(clf.grid_scores_)
print(clf.best_estimator_.n_features_)

ValueError: The total space of parameters 4 is smaller than n_iter=10. For exhaustive searches, use GridSearchCV.

In [21]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
iris = load_iris()
X, y = iris.data, iris.target
kbest = SelectKBest(f_classif)
pipeline = Pipeline([('kbest', kbest), ('lr', LogisticRegression())])
grid_search = GridSearchCV(pipeline, {'kbest__k': [1,2,3,4], 'lr__C': np.logspace(-10, 10, 5)})
grid_search.fit(X, y)


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('kbest', SelectKBest(k=10, score_func=<function f_classif at 0x0000014501273E18>)), ('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'lr__C': array([  1.00000e-10,   1.00000e-05,   1.00000e+00,   1.00000e+05,
         1.00000e+10]), 'kbest__k': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [25]:
#print(grid_search.grid_scores_)
print(grid_search.best_estimator_.n_features_)

AttributeError: 'Pipeline' object has no attribute 'n_features_'

In [24]:
print(grid_search.best_score_)

0.9666666666666667
